<center> <font size=6> <b> Table of Contents </b> </font> </center> 
<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

<center> <font size=5> <h1>Dependency</h1> </font> </center> 

This process depend from different software. Please instal them.
- Please install [psycopg2](http://initd.org/psycopg/) which is used to interact between the Jupyter notebook and a PostgreSql database. On Linux, this command-line should works: `pip install psycopg2`
- Please install [osm2pgsql](http://wiki.openstreetmap.org/wiki/Osm2pgsql#Installation) which is command-line based program used to import .osm file in a PostgreSql database. For Windows user, you could find the osm2pgsql executable. Go to [this website](https://ci.appveyor.com/project/openstreetmap/osm2pgsql/history) and find the latest 'artifact'. For Linux user, this command-line should works: `sudo apt-get install osm2pgsql`

<center> <font size=5> <h1>Define working environment</h1> </font> </center> 

The following cells are used to: 
- Import needed libraries
- Set the environment variables for Python, Anaconda, GRASS GIS and R statistical computing 
- Define the ["GRASSDATA" folder](https://grass.osgeo.org/grass73/manuals/helptext.html), the name of "location" and "mapset" where you will to work.

**Import libraries**

In [2]:
## Import libraries needed for setting parameters of operating system 
import os
import sys

## Import library for temporary files creation 
import tempfile 

## Import Numpy library
import numpy as np

## Import Psycopg2 library (interection with postgres database)
import psycopg2 as pg

## Import Pandas library (View and manipulaiton of tables)
import pandas as pd

## Import Subprocess + subprocess.call
import subprocess
from subprocess import call, Popen, PIPE, STDOUT

**Create dictionnary for user inputs**

In [3]:
## Define a empty dictionnary for saving user inputs
user={}

<center> <font size=5> <h3>Environment variables</h3> </font> </center> 

**Set 'Python' and 'GRASS GIS' environment variables**

Here, we set [the environment variables allowing to use of GRASS GIS](https://grass.osgeo.org/grass72/manuals/variables.html) inside this Jupyter notebook. Please change the directory path according to your own system configuration. Here after, possible paths are provided for different environment: 
- Windows7, using Anaconda2 and GRASS GIS 7.3svn standalone instal.
- Linux Mint Serena (18.1) and GRASS GIS 7.3. Suggestions about environmental variables for Linux can be found here : [1](https://code.google.com/archive/p/postgis-grass-r-py/wikis/0003_01_PythonForGrassGis.wiki), [2](https://grasswiki.osgeo.org/wiki/Working_with_GRASS_without_starting_it_explicitly#Python:_GRASS_GIS_7_without_existing_location_using_metadata_only)

In [4]:
### Define GRASS GIS environment variables for LINUX UBUNTU Mint 18.1 (Serena)
# Check is environmental variables exists and create them (empty) if not exists.
if not 'PYTHONPATH' in os.environ:
    os.environ['PYTHONPATH']=''
if not 'LD_LIBRARY_PATH' in os.environ:
    os.environ['LD_LIBRARY_PATH']=''
# Set environmental variables
os.environ['GISBASE'] = '/usr/lib/grass73'
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'bin')
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'script')
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'lib')
#os.environ['PATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'etc','python')
os.environ['PYTHONPATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'etc','python')
os.environ['PYTHONPATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'etc','python','grass')
os.environ['PYTHONPATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'etc','python','grass','script')
os.environ['PYTHONLIB'] = '/usr/lib/python2.7'
os.environ['LD_LIBRARY_PATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'lib')
os.environ['GIS_LOCK'] = '$$'
os.environ['GISRC'] = os.path.join(os.environ['HOME'],'.grass7','rc')

## Define GRASS-Python environment
sys.path.append(os.path.join(os.environ['GISBASE'],'etc','python'))

**Set environment variables for Osm2psgsql**

In [5]:
## Enter the path to the osm2pgsql folder
user["osm2pgsqlfolder"]="/usr/bin/osm2pgsql"

In [6]:
# Define the path to the osm2pgsql default.style file
user["stylefile"]="/usr/share/osm2pgsql/default_OSMmetadata.style"

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

<center> <font size=5> <h1>Define functions</h1> </font> </center> 

This section of the notebook is dedicated to defining functions which will then be called later in the script. If you want to create your own functions, define them here.

### Function for computing processing time

The "print_processing_time" is used to calculate and display the processing time for various stages of the processing chain. At the beginning of each major step, the current time is stored in a new variable, using [time.time() function](https://docs.python.org/2/library/time.html). At the end of the stage in question, the "print_processing_time" function is called and takes as argument the name of this new variable containing the recorded time at the beginning of the stage, and an output message.

In [7]:
## Import library for managing time in python
import time  

## Function "print_processing_time()" compute processing time and printing it.
# The argument "begintime" wait for a variable containing the begintime (result of time.time()) of the process for which to compute processing time.
# The argument "printmessage" wait for a string format with information about the process. 
def print_processing_time(begintime, printmessage):    
    endtime=time.time()           
    processtime=endtime-begintime
    remainingtime=processtime

    days=int((remainingtime)/86400)
    remainingtime-=(days*86400)
    hours=int((remainingtime)/3600)
    remainingtime-=(hours*3600)
    minutes=int((remainingtime)/60)
    remainingtime-=(minutes*60)
    seconds=round((remainingtime)%60,1)

    if processtime<60:
        finalprintmessage=str(printmessage)+str(seconds)+" seconds"
    elif processtime<3600:
        finalprintmessage=str(printmessage)+str(minutes)+" minutes and "+str(seconds)+" seconds"
    elif processtime<86400:
        finalprintmessage=str(printmessage)+str(hours)+" hours and "+str(minutes)+" minutes and "+str(seconds)+" seconds"
    elif processtime>=86400:
        finalprintmessage=str(printmessage)+str(days)+" days, "+str(hours)+" hours and "+str(minutes)+" minutes and "+str(seconds)+" seconds"
    
    return finalprintmessage

### Function for Postgresql database vaccum

In [8]:
# Do a VACUUM on the current Postgresql database
def vacuum(db):
    old_isolation_level = db.isolation_level
    db.set_isolation_level(0)
    query = "VACUUM"
    cur.execute(query)
    db.set_isolation_level(old_isolation_level)

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

<center> <font size=5> <h1>User inputs</h1> </font> </center> 

Here after:
- Enter the name of the location in which you want to work and its projection information in [EPSG code](http://spatialreference.org/ref/epsg/) format. Please note that the GRASSDATA folder and locations will be automatically created if not existing yet. If the location name already exists, the projection information will not be used.  
- Enter the name you want for the mapsets which will be used later for Unsupervised Segmentation Parameter Optimization (USPO), Segmentation and Classification steps.
- The environment variables of Postgresql will be defined. [More info here](https://www.postgresql.org/docs/9.3/static/libpq-envars.html).

In [9]:
## Enter the path to GRASSDATA folder
user["gisdb"] = "/media/tais/data/MAUPP/Landuse_mapping/City_block_extraction/Test_extraction_bloc/GRASS_DATA"
## Enter the name of the location (existing or for a new one)
user["location"] = "Ouaga_32630"
## Enter the EPSG code for this location 
user["locationepsg"] = "32630"
## Enter the name of the mapset to use
user["mapsetname"] = "my_mapset"

In [10]:
## Enter postgresqgl host
user["host"] = "localhost"
## Enter DB port
user["port"] = "5432"
## Enter the postgresqgl username
user["user"] = "tais"
## Enter postgresqgl Password
user["password"] = "tais"
## Enter postgresqgl schema
user["schema"] = "public"
## Enter the name of the new postgresqgl database
user["dbname"] = "urbanblock_new"

In [11]:
os.environ['PGHOST'] = user["host"]
os.environ['PGPORT'] = user["port"]
os.environ['PGUSER'] = user["user"]
os.environ['PGPASSWORD'] = user["password"]
os.environ['PGDATABASE'] = user["dbname"]

Here after:
- Enter the path to the folder where .osm file covering your area of interest will be downloaded. 
OSM data downloading is automated in this script. In case you would manage yourself the retrieving of OSM data, please read the [official wiki page](http://wiki.openstreetmap.org/wiki/Downloading_data) for that purpose. In tha case some parts of tis notebook should be adapted.

In [12]:
## Enter the path to the .osm file
user["osmfolder"]="/media/tais/data/MAUPP/Donnees_brutes/OSM/Ouagadougou"
## Enter the prefix you want to be used in the PostGIS DB
user["prefixosm"]="ouaga_osm"

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

<center> <font size=5> <h1>Import data in PostGIS databse</h1> </font>  </center>

In [13]:
## Saving current time for processing time management
begintime_blockextraction_full=time.time()

# Create PostGis database

## Define name of database and tables

In [14]:
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Connect to postgres database
db=None
db=pg.connect(dbname='postgres', user=user["user"], password=user["password"], host=user["host"])
cur=db.cursor()
# Allow to create a new database
db.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
# Drop the database if it exists, and create a new one 
#cur.execute('DROP DATABASE IF EXISTS ' + user["dbname"]) #Comment this to avoid deleting existing DB
cur.execute('CREATE DATABASE ' + user["dbname"])
# Make the changes to the database persistent
db.commit()
# Close connection with database
cur.close()
db.close()

ProgrammingError: ERREUR:  la base de données « urbanblock_new » existe déjà


In [15]:
# Connect to the new database
db=None
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
cur=db.cursor()
# Create the postgis extension
cur.execute('CREATE EXTENSION IF NOT EXISTS postgis')
# Make the changes to the database persistent
db.commit()
# Drop table if exists:
cur.execute('CREATE EXTENSION IF NOT EXISTS postgis_topology')
# Make the changes to the database persistent
db.commit()
# Make vaccum
vacuum(db)
# Close connection with database
cur.close()
db.close()

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Import additional vector layers (shapefile) in PostGis DB

The following part used the "shp2pgsql" program which should already be installed since postgis extension have been created in postgresql. See [this quick guide](http://www.bostongis.com/pgsql2shp_shp2pgsql_quickguide.bqg) for more information. 

## Morphological zones

In [16]:
#### Delineation of morphological zones
# Path to shapefile
pathtofile1="/media/tais/data/MAUPP/WorldView3_Ouagadougou/Decoupage_Morpho/Shape_limites_zonesmorpho_Ouaga/Orthorectified/Refined_version/Zone_morpho_Ouaga_refined.shp"
## Enter postgresqgl table's name
user["morphotable"] = "morpho_delineation"

In [51]:
cmd="shp2pgsql "
cmd+="-s "+user["locationepsg"]+":3857 "
cmd+="-d -I "
cmd+=pathtofile1+" "
cmd+=user["schema"]+"."+user["morphotable"]
cmd+=" | psql"
os.system(cmd)

0

**UPDATE last created PostGis table to ensure all geometries are valid**

In [52]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

# Query
query="UPDATE "+user["schema"]+"."+user["morphotable"]+" \
SET geom = ST_Multi(ST_CollectionExtract(ST_MakeValid(geom), 3)) \
WHERE ST_IsValid(geom) is not True"
# Execute the CREATE TABLE query 
cur.execute(query)
# Make the changes to the database persistent
db.commit()
   
# Close cursor and communication with the database
cur.close()
db.close()

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

## Area Of Interest of the city (AOI)

In [17]:
## Enter postgresqgl table's name
user["AOI"] = "AOI"

In [54]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()
# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+user["AOI"])
# Make the changes to the database persistent
db.commit()
# Make vaccum
vacuum(db)

In [55]:
#### Create a polygon with the AOI from the morphological zones
# Subquery
subquery1="SELECT ST_MakePolygon(ST_ExteriorRing(ST_Union(ST_MakeValid(geom)))) AS the_geom \
FROM "+user["schema"]+"."+user["morphotable"]
# Query
query="CREATE TABLE "+user["schema"]+"."+user["AOI"]+" AS ("
query+=subquery1+")"
# Execute the CREATE TABLE query 
cur.execute(query)
# Make the changes to the database persistent
db.commit()

# Create spatial index on the geometry column:
cur.execute("CREATE INDEX "+user["AOI"]+"_gix"+" ON "+user["schema"]+"."+user["AOI"]+" USING GIST (the_geom)")
# Make the changes to the database persistent
db.commit() 

# Close cursor and communication with the database
cur.close()
db.close()

In [56]:
# Query to find the number of row in the sample table
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Query to find the number of row in the  table
query="SELECT count(*) as nbr FROM "+user["schema"]+"."+user["AOI"] 
# Execute query through panda
df=pd.read_sql(query, db)
# Save the number of items a variable
nbr_blocks=list(df['nbr'])[0]
# Print
print "Number of items : "+str(nbr_blocks)

Number of items : 1


**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

## Tiles covering the extent of the AOI

In [18]:
## Enter postgresqgl table's name
user["tile"] = "tile"

In [58]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

In [59]:
# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+user["tile"])
# Make the changes to the database persistent
db.commit()
# Make vaccum
vacuum(db)

In [60]:
### Save the previous subquery1 results in a new table
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

subquery1="WITH \
first_BBox AS(\
SELECT ST_Segmentize(ST_Envelope(the_geom),(ST_Perimeter(the_geom)/(4*2))) as the_geom \
FROM public.aoi),\
 \
first_subdivide AS(\
SELECT ST_Subdivide(the_geom,8) AS the_geom \
FROM first_BBox),\
 \
second_BBox AS(\
SELECT ST_Segmentize(ST_Envelope(the_geom),(ST_Perimeter(the_geom)/(4*2))) as the_geom \
FROM first_subdivide)\
 \
SELECT ST_Subdivide(the_geom,8) AS the_geom \
FROM second_BBox"

# Query
query="CREATE TABLE "+user["schema"]+"."+user["tile"]+" AS ("
query+=subquery1+")"

# Execute the CREATE TABLE query 
cur.execute(query)
# Make the changes to the database persistent
db.commit()
# Close cursor and communication with the database
cur.close()
db.close()

Add a 'gid' SERIAL primary key and create a spatial index.

In [61]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()
# Query
query="ALTER TABLE "+user["schema"]+"."+user["tile"]+" ADD COLUMN gid SERIAL PRIMARY KEY"
# Execute the CREATE TABLE query 
cur.execute(query)
# Make the changes to the database persistent
db.commit()

# Create spatial index:
cur.execute("CREATE INDEX "+user["tile"]+"_gix"+" ON "+user["schema"]+"."+user["tile"]+" USING GIST (the_geom)")
# Make the changes to the database persistent
db.commit() 

# Close cursor and communication with the database
cur.close()
db.close()

In [19]:
# Query to find the number of row in the sample table
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Query to find the number of row in the  table
query="SELECT count(*) as nbr FROM "+user["schema"]+"."+user["tile"] 
# Execute query through panda
df=pd.read_sql(query, db)
# Save the number of items a variable
nbr_tiles=list(df['nbr'])[0]
# Print
print "Number of items : "+str(nbr_tiles)

Number of items : 16


**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

### Get upper, lower, right and left coordinates of each tile (in WGS84).

NB: the following query works for North hemisphere, west longitudes. Adapt ST_Ymin,ST_Ymax,ST_Xmin,ST_Xmax according to your geographical region.

In [20]:
## Enter postgresqgl table's mane
user["tile_coord"] = "tile_coord"

In [68]:
### Save the previous subquery1 results in a new table
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

# Query
query="WITH \
wgs84geom AS(\
SELECT  gid, ST_Transform(the_geom, 4326) AS the_geom \
FROM "+user["schema"]+"."+user["tile"]+") \
\
SELECT gid, ST_Xmin(the_geom) AS west, ST_Xmax(the_geom) AS east, \
ST_Ymin(the_geom) AS south, ST_Ymax(the_geom) AS north \
FROM wgs84geom"

# Execute query through panda
df_tiles_coord=pd.read_sql(query, db)
# Close cursor and communication with the database
cur.close()
db.close()
# Show dataframe
df_tiles_coord.head(50)

,gid,west,east,south,north
0,1,-1.685413,-1.611509,12.222975,12.288475
1,2,-1.685413,-1.611509,12.288475,12.353959
2,3,-1.611509,-1.537604,12.222975,12.288475
3,4,-1.611509,-1.537604,12.288475,12.353959
4,5,-1.685413,-1.611509,12.353959,12.419426
5,6,-1.685413,-1.611509,12.419426,12.484878
6,7,-1.611509,-1.537604,12.353959,12.419426
7,8,-1.611509,-1.537604,12.419426,12.484878
8,9,-1.537604,-1.463699,12.222975,12.288475
9,10,-1.537604,-1.463699,12.288475,12.353959


**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

## Ajust the tiles to the AOI (ST_Intersection).

In [21]:
## Enter postgresqgl table's name
user["aoi_tile"] = "AOI_tile"

In [66]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+user["aoi_tile"])
# Make the changes to the database persistent
db.commit()
# Make vaccum
vacuum(db)

# Subquery with the new 'the_geom' column resulting from ST_Interection
subquery="WITH aoi_buffered AS (\
SELECT ST_Buffer(the_geom,0.01) AS the_geom \
FROM "+user["schema"]+"."+user["AOI"]+") \
\
SELECT \
tile.gid AS gid, \
ST_Intersection(aoi.the_geom, tile.the_geom) as the_geom \
FROM aoi_buffered AS aoi,"+user["schema"]+"."+user["tile"]+" AS tile \
WHERE ST_Intersects(aoi.the_geom, tile.the_geom)"

# Query
query="CREATE TABLE "+user["schema"]+"."+user["aoi_tile"]+" AS ("
query+=subquery+")"

# Execute the CREATE TABLE query 
cur.execute(query)
# Make the changes to the database persistent
db.commit()
# Close cursor and communication with the database
cur.close()
db.close()

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

## OpenStreetMap data

The trick to extract polygons from lines was found [here](http://gis.stackexchange.com/questions/83/separate-polygons-based-on-intersection-using-postgis).

### Download OpenStreetMap data using Xapi

More information about Xapi can be found [here](http://wiki.openstreetmap.org/wiki/Xapi).

In [69]:
import datetime
import urllib

## Saving current time for processing time management
begintime_download=time.time()
  
## Save the suffix corresponding to the date of today
now=datetime.datetime.now()
date_suffix=str(now.year)+str(now.month)+str(now.day)

for tilenum in range(1,nbr_tiles+1):
    north_coord=round(list(df_tiles_coord.loc[tilenum-1,['north']])[0],6)
    south_coord=round(list(df_tiles_coord.loc[tilenum-1,['south']])[0],6)
    west_coord=round(list(df_tiles_coord.loc[tilenum-1,['west']])[0],6)
    east_coord=round(list(df_tiles_coord.loc[tilenum-1,['east']])[0],6)
    print "OSM data will be downloaded for tile n°"+str(tilenum)+" (W:"+str(west_coord)+" S:"+str(south_coord)+" E:"+str(east_coord)+" N:"+str(north_coord)+")"
    
    osm_api_base_url="http://www.overpass-api.de/api/xapi?*"
    osm_api_query_url=osm_api_base_url+"[bbox="+str(west_coord)+","+str(south_coord)+","+str(east_coord)+","+str(north_coord)+"][@meta]"
    
    #osm_file=user["osmfolder"]+"/"+user["prefixosm"]+"_"+str(now.year)+str(now.month)+str(now.day)+"_"+str(tilenum)+".osm"
    osm_file=os.path.join(user["osmfolder"],user["prefixosm"]+"_"+date_suffix+"_"+str(tilenum)+".osm")

    urllib.urlretrieve(osm_api_query_url, osm_file)
    print osm_api_query_url
    
## Print
print print_processing_time(begintime_download, "OSM Data downloaded in ")

OSM data will be downloaded for tile n°1 (W:-1.685413 S:12.222975 E:-1.611509 N:12.288475)
http://www.overpass-api.de/api/xapi?*[bbox=-1.685413,12.222975,-1.611509,12.288475][@meta]
OSM data will be downloaded for tile n°2 (W:-1.685413 S:12.288475 E:-1.611509 N:12.353959)
http://www.overpass-api.de/api/xapi?*[bbox=-1.685413,12.288475,-1.611509,12.353959][@meta]
OSM data will be downloaded for tile n°3 (W:-1.611509 S:12.222975 E:-1.537604 N:12.288475)
http://www.overpass-api.de/api/xapi?*[bbox=-1.611509,12.222975,-1.537604,12.288475][@meta]
OSM data will be downloaded for tile n°4 (W:-1.611509 S:12.288475 E:-1.537604 N:12.353959)
http://www.overpass-api.de/api/xapi?*[bbox=-1.611509,12.288475,-1.537604,12.353959][@meta]
OSM data will be downloaded for tile n°5 (W:-1.685413 S:12.353959 E:-1.611509 N:12.419426)
http://www.overpass-api.de/api/xapi?*[bbox=-1.685413,12.353959,-1.611509,12.419426][@meta]
OSM data will be downloaded for tile n°6 (W:-1.685413 S:12.419426 E:-1.611509 N:12.484878)

### Import OpenStreetMap layers in PostGis DB

Here after, the OpenStreetMap data will be imported in the Postgresql Database. [Osm2pgsql](http://wiki.openstreetmap.org/wiki/Osm2pgsql) is used for this purpose. Informations can be found in [this github repository](https://github.com/openstreetmap/osm2pgsql). If you are working on Windows, this tool cand be found on [this website](https://ci.appveyor.com/project/openstreetmap/osm2pgsql/history) (please look for the latest 'artifact').

The parameter to be used are explained [here](http://www.volkerschatz.com/net/osm/osm2pgsql-usage.html).

In [22]:
#### Please choose the version of .osm dataset to be used. 
# If you want to use the dataset which has just been downloaded (previous step), please use 'yes' value for 
# the "use_today_dataset" variable. If not, please use 'no' and update the "date_suffix" variable according
# to the version of the dataset you want to be used (avoid zero, e.g. 201756 for the 6 of May 2017)
use_today_dataset='no'
if use_today_dataset=='no':
    date_suffix='2017816'

In [76]:
## Loop through the stack of tiles
for tilenum in range(1,nbr_tiles+1):
    ## Call osm2pgsql as a subprocess
    p=Popen(['osm2pgsql','-c','-d',user["dbname"],
              '-U',user["user"],
              '-H',user["host"],
              '--extra-attributes',
              '-E','3857',
              '-p',user["prefixosm"]+"_"+date_suffix+"_"+str(tilenum),
              '-S',user["stylefile"],
              os.path.join(user["osmfolder"],user["prefixosm"]+"_"+date_suffix+"_"+str(tilenum)+".osm")],
            env={'PGPASS': user["password"]},
            stdin=PIPE,stdout=PIPE,stderr=STDOUT)
    stdout,stderr=p.communicate()
    if stdout:
        print stdout
    if stderr:
        print ("ERROR MESSAGE:\n"+stderr)

osm2pgsql SVN version 0.88.1 (64bit id space)

Unknown flag 'nocolumn' line 161, ignored
Unknown flag 'nocolumn' line 162, ignored
Unknown flag 'nocolumn' line 163, ignored
Unknown flag 'nocolumn' line 164, ignored
Unknown flag 'nocolumn' line 165, ignored
Unknown flag 'nocolumn' line 166, ignored
Using built-in tag processing pipeline
Using projection SRS 3857 (EPSG:3857)
Setting up table: ouaga_osm_2017816_1_point
Setting up table: ouaga_osm_2017816_1_line
Setting up table: ouaga_osm_2017816_1_polygon
Setting up table: ouaga_osm_2017816_1_roads
Allocating memory for dense node cache
Allocating dense node cache in one big chunk
Allocating memory for sparse node cache
Sharing dense sparse
Node-cache: cache=800MB, maxblocks=102400*8192, allocation method=3
Mid: Ram, scale=100

Reading in file: /media/tais/data/MAUPP/Donnees_brutes/OSM/Ouagadougou/ouaga_osm_2017816_1.osm
StartElement: Unknown element name: note
Unknown node type 3
EndElement: Unknown element name: note
StartElement: Unkn

osm2pgsql SVN version 0.88.1 (64bit id space)

Unknown flag 'nocolumn' line 161, ignored
Unknown flag 'nocolumn' line 162, ignored
Unknown flag 'nocolumn' line 163, ignored
Unknown flag 'nocolumn' line 164, ignored
Unknown flag 'nocolumn' line 165, ignored
Unknown flag 'nocolumn' line 166, ignored
Using built-in tag processing pipeline
Using projection SRS 3857 (EPSG:3857)
Setting up table: ouaga_osm_2017816_4_point
Setting up table: ouaga_osm_2017816_4_line
Setting up table: ouaga_osm_2017816_4_polygon
Setting up table: ouaga_osm_2017816_4_roads
Allocating memory for dense node cache
Allocating dense node cache in one big chunk
Allocating memory for sparse node cache
Sharing dense sparse
Node-cache: cache=800MB, maxblocks=102400*8192, allocation method=3
Mid: Ram, scale=100

Reading in file: /media/tais/data/MAUPP/Donnees_brutes/OSM/Ouagadougou/ouaga_osm_2017816_4.osm
StartElement: Unknown element name: note
Unknown node type 3
EndElement: Unknown element name: note
StartElement: Unkn

osm2pgsql SVN version 0.88.1 (64bit id space)

Unknown flag 'nocolumn' line 161, ignored
Unknown flag 'nocolumn' line 162, ignored
Unknown flag 'nocolumn' line 163, ignored
Unknown flag 'nocolumn' line 164, ignored
Unknown flag 'nocolumn' line 165, ignored
Unknown flag 'nocolumn' line 166, ignored
Using built-in tag processing pipeline
Using projection SRS 3857 (EPSG:3857)
Setting up table: ouaga_osm_2017816_7_point
Setting up table: ouaga_osm_2017816_7_line
Setting up table: ouaga_osm_2017816_7_polygon
Setting up table: ouaga_osm_2017816_7_roads
Allocating memory for dense node cache
Allocating dense node cache in one big chunk
Allocating memory for sparse node cache
Sharing dense sparse
Node-cache: cache=800MB, maxblocks=102400*8192, allocation method=3
Mid: Ram, scale=100

Reading in file: /media/tais/data/MAUPP/Donnees_brutes/OSM/Ouagadougou/ouaga_osm_2017816_7.osm
StartElement: Unknown element name: note
Unknown node type 3
EndElement: Unknown element name: note
StartElement: Unkn

osm2pgsql SVN version 0.88.1 (64bit id space)

Unknown flag 'nocolumn' line 161, ignored
Unknown flag 'nocolumn' line 162, ignored
Unknown flag 'nocolumn' line 163, ignored
Unknown flag 'nocolumn' line 164, ignored
Unknown flag 'nocolumn' line 165, ignored
Unknown flag 'nocolumn' line 166, ignored
Using built-in tag processing pipeline
Using projection SRS 3857 (EPSG:3857)
Setting up table: ouaga_osm_2017816_10_point
Setting up table: ouaga_osm_2017816_10_line
Setting up table: ouaga_osm_2017816_10_polygon
Setting up table: ouaga_osm_2017816_10_roads
Allocating memory for dense node cache
Allocating dense node cache in one big chunk
Allocating memory for sparse node cache
Sharing dense sparse
Node-cache: cache=800MB, maxblocks=102400*8192, allocation method=3
Mid: Ram, scale=100

Reading in file: /media/tais/data/MAUPP/Donnees_brutes/OSM/Ouagadougou/ouaga_osm_2017816_10.osm
StartElement: Unknown element name: note
Unknown node type 3
EndElement: Unknown element name: note
StartElement:

osm2pgsql SVN version 0.88.1 (64bit id space)

Unknown flag 'nocolumn' line 161, ignored
Unknown flag 'nocolumn' line 162, ignored
Unknown flag 'nocolumn' line 163, ignored
Unknown flag 'nocolumn' line 164, ignored
Unknown flag 'nocolumn' line 165, ignored
Unknown flag 'nocolumn' line 166, ignored
Using built-in tag processing pipeline
Using projection SRS 3857 (EPSG:3857)
Setting up table: ouaga_osm_2017816_13_point
Setting up table: ouaga_osm_2017816_13_line
Setting up table: ouaga_osm_2017816_13_polygon
Setting up table: ouaga_osm_2017816_13_roads
Allocating memory for dense node cache
Allocating dense node cache in one big chunk
Allocating memory for sparse node cache
Sharing dense sparse
Node-cache: cache=800MB, maxblocks=102400*8192, allocation method=3
Mid: Ram, scale=100

Reading in file: /media/tais/data/MAUPP/Donnees_brutes/OSM/Ouagadougou/ouaga_osm_2017816_13.osm
StartElement: Unknown element name: note
Unknown node type 3
EndElement: Unknown element name: note
StartElement:

osm2pgsql SVN version 0.88.1 (64bit id space)

Unknown flag 'nocolumn' line 161, ignored
Unknown flag 'nocolumn' line 162, ignored
Unknown flag 'nocolumn' line 163, ignored
Unknown flag 'nocolumn' line 164, ignored
Unknown flag 'nocolumn' line 165, ignored
Unknown flag 'nocolumn' line 166, ignored
Using built-in tag processing pipeline
Using projection SRS 3857 (EPSG:3857)
Setting up table: ouaga_osm_2017816_16_point
Setting up table: ouaga_osm_2017816_16_line
Setting up table: ouaga_osm_2017816_16_polygon
Setting up table: ouaga_osm_2017816_16_roads
Allocating memory for dense node cache
Allocating dense node cache in one big chunk
Allocating memory for sparse node cache
Sharing dense sparse
Node-cache: cache=800MB, maxblocks=102400*8192, allocation method=3
Mid: Ram, scale=100

Reading in file: /media/tais/data/MAUPP/Donnees_brutes/OSM/Ouagadougou/ouaga_osm_2017816_16.osm
StartElement: Unknown element name: note
Unknown node type 3
EndElement: Unknown element name: note
StartElement:

Here after, the osm table "poin" and "roads" will be droped as they will not be used in our processings.

In [77]:
## Drop tables "point" and "roads" which will not be used in this script

# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

for tilenum in range(1,nbr_tiles+1):
    # Drop table if exists:
    cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+"\
    "+user["prefixosm"]+"_"+date_suffix+"_"+str(tilenum)+"_"+"point")

    # Drop table if exists:
    cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+"\
    "+user["prefixosm"]+"_"+date_suffix+"_"+str(tilenum)+"_"+"roads")
    
# Make the changes to the database persistent
db.commit()
# Make vaccum
vacuum(db)

Here after, the name of the osm table will be saved in dictionnaries.

In [23]:
## Define a empty dictionnary for saving osm line and polygon table names
osm_line={}
osm_polygon={}

## Loop through the stack of tiles
for tilenum in range(1,nbr_tiles+1):
    # Save the table postgis table name in the dictionnary
    osm_line["tile_"+str(tilenum)]=user["prefixosm"]+"_"+date_suffix+"_"+str(tilenum)+"_line"
    osm_polygon["tile_"+str(tilenum)]=user["prefixosm"]+"_"+date_suffix+"_"+str(tilenum)+"_polygon"
    
#### Display name of osm tables
for tile in osm_line.keys():
    print osm_line[tile]+"    "+osm_polygon[tile]

ouaga_osm_2017816_16_line    ouaga_osm_2017816_16_polygon
ouaga_osm_2017816_15_line    ouaga_osm_2017816_15_polygon
ouaga_osm_2017816_14_line    ouaga_osm_2017816_14_polygon
ouaga_osm_2017816_13_line    ouaga_osm_2017816_13_polygon
ouaga_osm_2017816_12_line    ouaga_osm_2017816_12_polygon
ouaga_osm_2017816_11_line    ouaga_osm_2017816_11_polygon
ouaga_osm_2017816_10_line    ouaga_osm_2017816_10_polygon
ouaga_osm_2017816_7_line    ouaga_osm_2017816_7_polygon
ouaga_osm_2017816_6_line    ouaga_osm_2017816_6_polygon
ouaga_osm_2017816_5_line    ouaga_osm_2017816_5_polygon
ouaga_osm_2017816_4_line    ouaga_osm_2017816_4_polygon
ouaga_osm_2017816_3_line    ouaga_osm_2017816_3_polygon
ouaga_osm_2017816_2_line    ouaga_osm_2017816_2_polygon
ouaga_osm_2017816_1_line    ouaga_osm_2017816_1_polygon
ouaga_osm_2017816_9_line    ouaga_osm_2017816_9_polygon
ouaga_osm_2017816_8_line    ouaga_osm_2017816_8_polygon


**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

<center> *-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*- </center> 

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Extract urban blocks for each tile

In [24]:
#### Save names of linestrings and blocks for each tile

## Define a empty dictionnary for saving osm line and polygon table names
linestrings={}
blocks={}
linestrings_snaped={}
blocks_snaped={}

for tile in osm_line.keys()[:]:
    linestrings[tile] = "linestrings"+"_"+str(tile)
    linestrings_snaped[tile] = "linestrings_snaped"+"_"+str(tile)
    blocks[tile] = "urban_blocks"+"_"+str(tile)
    blocks_snaped[tile] = "urban_blocks_snaped"+"_"+str(tile)
    print linestrings[tile]+" ; "+linestrings_snaped[tile]+" ; "+blocks[tile]+" ; "+blocks_snaped[tile]

linestrings_tile_16 ; linestrings_snaped_tile_16 ; urban_blocks_tile_16 ; urban_blocks_snaped_tile_16
linestrings_tile_15 ; linestrings_snaped_tile_15 ; urban_blocks_tile_15 ; urban_blocks_snaped_tile_15
linestrings_tile_14 ; linestrings_snaped_tile_14 ; urban_blocks_tile_14 ; urban_blocks_snaped_tile_14
linestrings_tile_13 ; linestrings_snaped_tile_13 ; urban_blocks_tile_13 ; urban_blocks_snaped_tile_13
linestrings_tile_12 ; linestrings_snaped_tile_12 ; urban_blocks_tile_12 ; urban_blocks_snaped_tile_12
linestrings_tile_11 ; linestrings_snaped_tile_11 ; urban_blocks_tile_11 ; urban_blocks_snaped_tile_11
linestrings_tile_10 ; linestrings_snaped_tile_10 ; urban_blocks_tile_10 ; urban_blocks_snaped_tile_10
linestrings_tile_7 ; linestrings_snaped_tile_7 ; urban_blocks_tile_7 ; urban_blocks_snaped_tile_7
linestrings_tile_6 ; linestrings_snaped_tile_6 ; urban_blocks_tile_6 ; urban_blocks_snaped_tile_6
linestrings_tile_5 ; linestrings_snaped_tile_5 ; urban_blocks_tile_5 ; urban_blocks_snaped

## Extract linestrings from different sources

**If you faced an "GEOSUnaryUnion: TopologyException: found non-noded intersection between" error, please change a bit the value of 'snaptogrid_param' (increase for to 0.07 or 0.10 for example)**

In [94]:
## Saving current time for processing time management
begintime_linestrings_total=time.time()

snaptogrid_param="0.04"

# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

for tile in osm_line.keys()[:]:
    ## Saving current time for processing time management
    begintime_linestrings_loop=time.time()
    print "Working on "+str(tile)

    ########## Combine all linestrings from both lines and polygons in a single table ##########
   
    # Drop table if exists:
    cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+linestrings[tile])
    # Make the changes to the database persistent
    db.commit()
    # Make vaccum
    vacuum(db)
    
    # Subquery for extraction of linestrings from different sources
    subquery1="WITH \
    current_tile AS (\
    SELECT the_geom FROM "+user["schema"]+"."+user["aoi_tile"]+" AS tile \
    WHERE gid="+tile[tile.index("_")+1:]+"), \
    \
    linestrings AS (\
    SELECT  ST_ExteriorRing(aoi.the_geom) AS the_geom \
    FROM "+user["schema"]+"."+user["AOI"]+" AS aoi \
    \
    UNION ALL \
    \
    SELECT ST_SnapToGrid(l.way,"+snaptogrid_param+") AS the_geom \
    FROM "+user["schema"]+"."+osm_line[tile]+" AS l, current_tile \
    WHERE (l.highway is not null OR l.waterway is not null OR l.railway is not null) \
    AND ST_Intersects(l.way, current_tile.the_geom) \
    \
    UNION ALL \
    \
    SELECT ST_SnapToGrid(ST_ExteriorRing((ST_DumpRings(morpho.geom)).geom),"+snaptogrid_param+") AS the_geom \
    FROM (SELECT (ST_Dump(zm.geom)).* FROM "+user["schema"]+"."+user["morphotable"]+" AS zm) AS morpho, current_tile \
    WHERE ST_Intersects(morpho.geom, current_tile.the_geom) \
    \
    UNION ALL \
    \
    SELECT ST_SnapToGrid(ST_ExteriorRing((ST_DumpRings(naturpoly.geom)).geom),"+snaptogrid_param+") AS the_geom \
    FROM (SELECT (ST_Dump(p.way)).* FROM "+user["schema"]+"."+osm_polygon[tile]+" AS p, current_tile  \
    WHERE p.natural is not null AND ST_Area(ST_Transform(p.way, "+user["locationepsg"]+"))>2500 \
    AND ST_Intersects(p.way, current_tile.the_geom)) AS naturpoly \
    \
    UNION ALL \
    \
    SELECT ST_SnapToGrid(ST_ExteriorRing((ST_DumpRings(amenities.geom)).geom),"+snaptogrid_param+") AS the_geom \
    FROM (SELECT (ST_Dump(p.way)).* FROM "+user["schema"]+"."+osm_polygon[tile]+" AS p, current_tile  \
    WHERE p.amenity IN ('college','school','university','clinic','hospital') \
    AND ST_Area(ST_Transform(p.way, "+user["locationepsg"]+"))>1000 \
    AND ST_Intersects(p.way, current_tile.the_geom)) AS amenities \
    \
    UNION ALL\
    \
    SELECT ST_SnapToGrid(ST_ExteriorRing((ST_DumpRings(landusepoly.geom)).geom),"+snaptogrid_param+") AS the_geom \
    FROM (SELECT (ST_Dump(p.way)).* FROM "+user["schema"]+"."+osm_polygon[tile]+" AS p, current_tile \
    WHERE (p.landuse is not null OR p.barrier is not null) \
    AND ST_Area(ST_Transform(p.way, "+user["locationepsg"]+"))>2500 \
    AND ST_Intersects(p.way, current_tile.the_geom)) AS landusepoly), \
    \
    tiled_linestrings AS (\
    SELECT CASE \
    WHEN ST_CoveredBy(ls.the_geom,tile.the_geom) \
    THEN ls.the_geom \
    ELSE ST_Intersection(ls.the_geom,tile.the_geom) \
    END As the_geom \
    FROM linestrings AS ls \
    INNER JOIN current_tile As tile \
    ON ST_Intersects(ls.the_geom,tile.the_geom)) \
    \
    SELECT (ST_Dump(the_geom)).geom AS the_geom \
    FROM (SELECT ST_Union(ST_MakeValid(the_geom)) AS the_geom FROM tiled_linestrings) AS noded_line"

    # Query
    query="CREATE TABLE "+user["schema"]+"."+linestrings[tile]+" AS ("
    query+=subquery1+")"
    # Execute the CREATE TABLE query 
    cur.execute(query)
    # Make the changes to the database persistent
    db.commit()
    
    ## Print
    print print_processing_time(begintime_linestrings_loop, "Linestrings of '"+str(tile)+"' extracted in ")
        
# Close cursor and communication with the database
cur.close()
db.close()
    
## Print
print print_processing_time(begintime_linestrings_total, "Process achieved in ")

Working on tile_8
Linestrings of 'tile_8' extracted in 30.7 seconds
Process achieved in 30.7 seconds


## Merge all tiles together

In [25]:
## Enter postgresqgl table's name
user["linestrings_all"] = "linestrings_all"

In [46]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

In [47]:
## Saving current time for processing time management
begintime_tilemerging=time.time()
print "Going to merge all tiles together"

# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+user["linestrings_all"])
# Make the changes to the database persistent
db.commit()
# Make vaccum
vacuum(db)

# Subquery
subquery="WITH \
merged_tiles AS(\
SELECT all_tiles.the_geom AS the_geom \
FROM ("
for tile in linestrings.keys():
    if tile not in linestrings.keys()[-1:]:
        subquery+="SELECT * FROM "+user["schema"]+"."+linestrings[tile]+" \
                    UNION ALL \
                    "
    else :
        subquery+="SELECT * FROM "+user["schema"]+"."+linestrings[tile]+") AS all_tiles) "
        
subquery+="\
SELECT the_geom FROM merged_tiles"

# Query
query="CREATE TABLE "+user["schema"]+"."+user["linestrings_all"]+" AS ("
query+=subquery+")"

#print query 
# Execute the CREATE TABLE query 
cur.execute(query)
# Make the changes to the database persistent
db.commit()

# Add a SERIAL PRIMARY KEY on the table 
cur.execute("ALTER TABLE "+user["schema"]+"."+user["linestrings_all"]+" ADD COLUMN gid SERIAL PRIMARY KEY")
# Make the changes to the database persistent
db.commit()

## Print
print print_processing_time(begintime_tilemerging, "Linestrings from all tiles merged in ")

Going to merge all tiles together
Linestrings from all tiles merged in 0.7 seconds


In [48]:
# Close cursor and communication with the database
cur.close()
db.close()

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

### Use PostGis topology to clean the linestrings network (snapping-like)

The trick was found [here](http://blog.mathieu-leplatre.info/use-postgis-topologies-to-clean-up-road-networks.html).

In [49]:
# Set the tolerance for snapping (in meters)
tolerance=7

In [26]:
user["linestrings_all_snaped"]="linestrings_all_snaped"

In [51]:
## Saving current time for processing time management
begintime_linestrings=time.time()

# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

########## Clean the linestrings network using PostGis topology fonctions ##########
topo_layer="linestring_topo_test"

# Query
query="SELECT topology.DropTopology('"+topo_layer+"') \
WHERE EXISTS (SELECT * FROM topology.topology WHERE name='"+topo_layer+"')"
# Drop table if exists:
cur.execute(query)
# Make the changes to the database persistent
db.commit()

# Query
query="SELECT topology.CreateTopology('"+topo_layer+"', 3857, "+str(tolerance)+") \
WHERE NOT EXISTS (SELECT * FROM topology.topology WHERE name='"+topo_layer+"')"
# Drop table if exists:
cur.execute(query)
# Make the changes to the database persistent
db.commit()

# Query
query="SELECT topology.AddTopoGeometryColumn(\
'"+topo_layer+"','"+user["schema"]+"','"+user["linestrings_all"]+"','topo_geom','LINESTRING')"
# Drop table if exists:
cur.execute(query)
# Make the changes to the database persistent
db.commit()

# Query
query="\
DO $$DECLARE r record;\
BEGIN \
  FOR r IN SELECT * FROM "+user["schema"]+"."+user["linestrings_all"]+" LOOP \
    BEGIN \
      UPDATE "+user["schema"]+"."+user["linestrings_all"]+" \
        SET topo_geom = topology.toTopoGeom(the_geom, '"+topo_layer+"', 1, "+str(tolerance)+") \
      WHERE the_geom = r.the_geom; \
    EXCEPTION \
      WHEN OTHERS THEN \
        RAISE WARNING 'Loading of record % failed: %', r.the_geom, SQLERRM; \
    END; \
  END LOOP; \
END$$;"
# Drop table if exists:
cur.execute(query)
# Make the changes to the database persistent
db.commit()
# Make vaccum
vacuum(db)

# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+user["linestrings_all_snaped"])
# Make the changes to the database persistent
db.commit()
# Make vaccum
vacuum(db)

# Subquery
subquery="WITH \
valid_linestring AS(\
SELECT ST_MakeValid((ST_Dump(topo_geom::geometry)).geom) AS the_geom \
FROM "+user["schema"]+"."+user["linestrings_all"]+"), \
\
noded_linestring AS(\
SELECT (ST_Dump(ST_Union(the_geom))).geom AS the_geom FROM valid_linestring) \
\
SELECT * FROM noded_linestring"
# Query
query="CREATE TABLE "+user["schema"]+"."+user["linestrings_all_snaped"]+" AS ("
query+=subquery+")"

# Execute the query
cur.execute(query)
# Make the changes to the database persistent
db.commit() 
    
## Print
print print_processing_time(begintime_linestrings, "Process achieved in ")

Process achieved in 15 hours and 35 minutes and 55.7 seconds


### Polygonize linestrings network to create urban blocks

In [27]:
## Enter postgresqgl table's name
user["street_blocks_with_artifacts"] = "street_blocks_with_artifacts"

In [53]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

In [54]:
## Saving current time for processing time management
begintime_polygonize=time.time()

# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+user["street_blocks_with_artifacts"])
# Make the changes to the database persistent
db.commit()
# Make vaccum
vacuum(db)

# Subquery for polygonization of mutlilinestrings
subquery1="WITH \
multilinestring AS (\
SELECT ST_Multi(the_geom) AS the_geom \
FROM (SELECT ST_Collect(the_geom) AS the_geom \
      FROM "+user["schema"]+"."+user["linestrings_all_snaped"]+") As GeomCollection), \
\
valid_polygons AS (\
SELECT ST_CollectionExtract((ST_Dump(ST_MakeValid(the_geom))).geom,3) AS the_geom \
FROM (SELECT DISTINCT ST_CollectionExtract(((ST_Dump(ST_Polygonize(the_geom))).geom),3) AS the_geom \
FROM multilinestring) AS extracted_polygons) \
\
SELECT the_geom FROM valid_polygons"

# Query
query="CREATE TABLE "+user["schema"]+"."+user["street_blocks_with_artifacts"]+" AS ("
query+=subquery1+")"
# Execute the CREATE TABLE query 
cur.execute(query)
# Make the changes to the database persistent
db.commit()
# Add a SERIAL PRIMARY KEY on the table 
cur.execute("ALTER TABLE "+user["schema"]+"."+user["street_blocks_with_artifacts"]+" \
ADD COLUMN gid SERIAL PRIMARY KEY")
# Make the changes to the database persistent
db.commit()
    
## Print
print print_processing_time(begintime_polygonize, "Process achieved in ")

Process achieved in 5.8 seconds


In [55]:
# Query to find the number of row in the  table
query="SELECT count(*) as nbr FROM "+user["schema"]+"."+user["street_blocks_with_artifacts"]
# Execute query through panda
df=pd.read_sql(query, db)
# Save the number of items a variable
nbr_tiles=list(df['nbr'])[0]
# Print
print "Number of blocks extracted : "+str(nbr_tiles)

Number of blocks extracted : 47367


In [56]:
# Close cursor and communication with the database
cur.close()
db.close()

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

<center> *-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*- </center> 

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Remove artifact polygons

## Defining functions

In [36]:
def find_nbrrowm(db, schema, table):
    # Query to find the number of row in the  table
    query="SELECT count(*) as nbr FROM "+schema+"."+table
    # Execute query through panda
    df=pd.read_sql(query, db)
    # Save the number of items a variable
    nbr_row=list(df['nbr'])[0]
    # Return
    return nbr_row

In [37]:
def add_area_perimeter_compactness(db, schema, table):
    # Open a cursor to perform database operations
    cur=db.cursor()
    # Add columns if not exist
    cur.execute("ALTER TABLE "+schema+"."+table+" \
    ADD COLUMN IF NOT EXISTS area double precision, \
    ADD COLUMN IF NOT EXISTS perimeter double precision, \
    ADD COLUMN IF NOT EXISTS compactness double precision")
    # Make the changes to the database persistent
    db.commit()
    # Update columns with values
    cur.execute("UPDATE "+schema+"."+table+" \
    SET area=ST_Area(ST_Transform(the_geom,"+user["locationepsg"]+")), \
    perimeter=ST_Perimeter(ST_Transform(the_geom,"+user["locationepsg"]+")),\
    compactness=(ST_Perimeter(ST_Transform(the_geom,"+user["locationepsg"]+"))/sqrt(\
    ST_Area(ST_Transform(the_geom,"+user["locationepsg"]+"))))")
    # Make the changes to the database persistent
    db.commit()

In [90]:
def add_morphoid_morphotype(db, schema, streetblocktable, morphotable):
    # Open a cursor to perform database operations
    cur=db.cursor()
    # Add columns if not exist
    cur.execute("ALTER TABLE "+schema+"."+streetblocktable+" \
    ADD COLUMN IF NOT EXISTS morpho_id integer, \
    ADD COLUMN IF NOT EXISTS morpho_type varchar(10) ")
    # Make the changes to the database persistent
    db.commit()
    # Update columns with values
    cur.execute("UPDATE "+schema+"."+streetblocktable+" AS a \
    SET morpho_id=b.gid \
    FROM "+schema+"."+morphotable+" AS b \
    WHERE ST_Within(ST_PointOnSurface(a.the_geom),b.geom)")
    # Make the changes to the database persistent
    db.commit()   
    # Update columns with values
    cur.execute("UPDATE "+schema+"."+streetblocktable+" AS a \
    SET morpho_type=b.type \
    FROM "+schema+"."+morphotable+" AS b \
    WHERE ST_Within(ST_PointOnSurface(a.the_geom),b.geom)")
    # Make the changes to the database persistent
    db.commit()  

In [135]:
def find_artifact(db, schema, table):
    ## Enter postgresqgl table's mane
    temp_join = "temp_join"

    # Open a cursor to perform database operations
    cur=db.cursor()

    #### Identify the artifact polygon (identified by selection on area and perimeter) 
    #### and all polygons non-artifact polygons touching them.

    # Drop table if exists:
    cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+temp_join)
    # Make the changes to the database persistent
    db.commit()

    # Subquery
    subquery1="WITH \
    artifacts AS (\
    SELECT * FROM "+schema+"."+table+" AS p \
    WHERE (p.compactness>6.5 AND p.area<10000) OR (p.area<2200) OR (p.compactness>15)), \
    \
    basepolygons AS (\
    SELECT * FROM "+schema+"."+table+" AS p \
    WHERE NOT EXISTS (SELECT a.gid FROM artifacts AS a WHERE p.gid=a.gid)) \
    \
    SELECT a.the_geom, a.gid AS artifact_gid, b.gid AS base_gid, a.morpho_id, a.morpho_type, \
    ST_Length(ST_CollectionExtract(ST_Intersection(a.the_geom, b.the_geom), 2)) AS length \
    FROM artifacts a INNER JOIN basepolygons b ON (ST_Touches(a.the_geom,b.the_geom) AND a.morpho_id=b.morpho_id) \
    WHERE ST_Length(ST_CollectionExtract(ST_Intersection(a.the_geom, b.the_geom), 2)) > 0 \
    ORDER BY a.gid"
    
    # Query
    query="CREATE TABLE "+schema+"."+temp_join+" AS ("
    query+=subquery1+")"

    # Execute the CREATE TABLE query 
    cur.execute(query)
    # Make the changes to the database persistent
    db.commit()

    #### For each artifact, keep only the record regarding to the longer shared lenght with basepolygon.

    # Set the name of the temporary layer
    global temp_maxborder
    temp_maxborder="temp_maxborder"

    # Drop table if exists:
    cur.execute("DROP TABLE IF EXISTS "+schema+"."+temp_maxborder)
    # Make the changes to the database persistent
    db.commit()

    # Subquery for extraction of linestrings from different sources
    subquery="WITH \
    max_value AS(\
    SELECT artifact_gid, max(length) AS shared_length \
    FROM "+schema+"."+temp_join+" \
    GROUP BY artifact_gid ORDER BY artifact_gid) \
    \
    SELECT a.* \
    FROM "+schema+"."+temp_join+" AS a \
    INNER JOIN max_value AS b \
    ON (a.length = b.shared_length AND a.artifact_gid = b.artifact_gid)"

    ## Query
    query="CREATE TABLE "+schema+"."+temp_maxborder+" AS ("
    query+=subquery+")"
    # Execute the CREATE TABLE query 
    cur.execute(query)
    # Make the changes to the database persistent
    db.commit()

    return find_nbrrowm(db, schema, temp_maxborder)

In [136]:
def remove_artifacts(db, schema, table):
    # Set the name of the temporary layer
    union_table="union_table"

    # Drop table if exists:
    cur.execute("DROP TABLE IF EXISTS "+schema+"."+union_table)
    # Make the changes to the database persistent
    db.commit()

    # Subquery for extraction of linestrings from different sources
    subquery="SELECT a.*, b.base_gid AS union_gid FROM "+schema+"."+table+" AS a \
    LEFT JOIN "+schema+"."+temp_maxborder+" AS b ON a.gid=b.artifact_gid"

    ## Query
    query="CREATE TABLE "+schema+"."+union_table+" AS ("
    query+=subquery+")"
    # Execute the CREATE TABLE query 
    cur.execute(query)
    # Make the changes to the database persistent
    db.commit()
    
    # Add a SERIAL PRIMARY KEY on the table 
    cur.execute("UPDATE "+schema+"."+union_table+" SET union_gid=gid WHERE union_gid IS NULL")
    # Make the changes to the database persistent
    db.commit()
    
    # Drop table if exists:
    cur.execute("DROP TABLE IF EXISTS "+schema+"."+table)
    # Make the changes to the database persistent
    db.commit()

    # Subquery for extraction of linestrings from different sources
    subquery="SELECT ST_Union(the_geom) AS the_geom, union_gid AS gid \
    FROM "+schema+"."+union_table+" \
    GROUP BY union_gid ORDER BY union_gid"

    ## Query
    query="CREATE TABLE "+schema+"."+table+" AS ("
    query+=subquery+")"
    # Execute the CREATE TABLE query 
    cur.execute(query)
    # Make the changes to the database persistent
    db.commit()

## Remove artifacts in a loop

In [137]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

**Copy the current street blocks in a temporary table for processing**

In [138]:
## Enter postgresqgl table's name
user["street_blocks_temp"] = "street_blocks_temp"

In [139]:
# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+user["street_blocks_temp"])
# Make the changes to the database persistent
db.commit()

# Subquery for extraction of linestrings from different sources
subquery="SELECT * FROM "+user["schema"]+"."+user["street_blocks_with_artifacts"]

## Query
query="CREATE TABLE "+user["schema"]+"."+user["street_blocks_temp"]+" AS ("
query+=subquery+")"
# Execute the CREATE TABLE query 
cur.execute(query)
# Make the changes to the database persistent
db.commit()

In [140]:
# Compute morphological metrics
add_area_perimeter_compactness(db,user["schema"],user["street_blocks_temp"])

In [ ]:
# Compute appartenance to morphological zones
add_morphoid_morphotype(db,user["schema"],user["street_blocks_temp"], user["morphotable"])

**Remove artifacts in a loop**

In [ ]:
## Saving current time for processing time management
begintime_cleaning=time.time()
 
# Compute the initial number of artifact
nbr_artifact=find_artifact(db,user["schema"],user["street_blocks_temp"])

# Remove artifacts in a loop
if nbr_artifact==0:
    print "There is no artifact to be removed"
else:
    nbr_loop=0
    while nbr_artifact > 0:
        nbr_loop+=1
        print "Pass n°"+str(nbr_loop)+".\n"
        print str(nbr_artifact)+" artifacts have to be removed.\n"
        remove_artifacts(db,user["schema"],user["street_blocks_temp"])
        print "Artifacts successfully removed.\n\n"
        add_area_perimeter_compactness(db,user["schema"], user["street_blocks_temp"])
        add_morphoid_morphotype(db,user["schema"],user["street_blocks_temp"], user["morphotable"])
        nbr_artifact=find_artifact(db,user["schema"], user["street_blocks_temp"])
    print "All artifacts were removed after "+str(nbr_loop)+" passes."
    
## Print
print print_processing_time(begintime_cleaning, "Artifacts where removed in ")

Pass n°1.

14169 artifacts have to be removed.

Artifacts successfully removed.


Pass n°2.

1130 artifacts have to be removed.

Artifacts successfully removed.


Pass n°3.

111 artifacts have to be removed.

Artifacts successfully removed.


Pass n°4.

18 artifacts have to be removed.

Artifacts successfully removed.


Pass n°5.

3 artifacts have to be removed.

Artifacts successfully removed.




In [ ]:
# Close cursor and communication with the database
cur.close()
db.close()

**Copy the result in the a new PostGis table**

In [ ]:
## Enter postgresqgl table's name
user["street_blocks"] = "street_blocks"

In [ ]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

In [ ]:
# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+user["street_blocks"])
# Make the changes to the database persistent
db.commit()

# Subquery for extraction of linestrings from different sources
subquery="SELECT * FROM "+user["schema"]+"."+user["street_blocks_temp"]

## Query
query="CREATE TABLE "+user["schema"]+"."+user["street_blocks"]+" AS ("
query+=subquery+")"
# Execute the CREATE TABLE query 
cur.execute(query)
# Make the changes to the database persistent
db.commit()

In [ ]:
# Close cursor and communication with the database
cur.close()
db.close()

## Compute some statistics about the street blocks

In [ ]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

In [ ]:
# Max number of records to return from the query
limitnumber=20
# Query
query="SELECT * FROM "+user["schema"]+"."+user["street_blocks"]+" LIMIT "+str(limitnumber)
# Execute query through panda
df=pd.read_sql(query, db)
# Show dataframe
df.head(limitnumber)

In [ ]:
# Query,
query="SELECT count(id), mean(area), stddev(area), mean(perimeter), stddev(perimeter) \
FROM "+user["schema"]+"."+user["street_blocks"]+" \
GROUP BY "
# Execute query through panda
df=pd.read_sql(query, db)
# Show dataframe
df.head(50)

## Remove all tables not needed anymore

## Remove temporary layer created during the process

In [137]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+temp_layer_touchingpolygon)
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+temp_layer_borderlength)
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+temp_layer_borderlength_max)
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+temp_layer_cleaned)
# Make the changes to the database persistent
db.commit()
# Make vaccum
vacuum(db)

# Close cursor and communication with the database
cur.close()
db.close()

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

<center> *-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*- </center> 

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

### Export urban blocks in Shapefile

dsjdsodlksdds

In [ ]:
## Shapefile of the morphological delineation
pathtofile1="F:\\MAUPP\\Landuse_mapping\\City_block_extraction\\Test_extraction_bloc\\Data\\Zone_morpho_Ouaga_ajusted_ortho.shp"

In [ ]:
## Build the osm2pgsql command-line
cmdline="set PGPASSWORD="+user["password"]+"\n"
cmdline+="pgsql2shp -s -d -I"+" "
cmdline+=pathtofile1+" "+user["schema"]+"."+user["blocks"]+" "
cmdline+="|"+" "
cmdline+="psql -d "+user["dbname"]+" -h "+user["host"]+" -U "+user["user"]
## Create temp bash file for osm2pgsql
outputcsv=tempfile.gettempdir()+"\\tmp_bash.bat" # Define the csv output file name
f = open(outputcsv, 'w')
f.write(cmdline)
f.close()

print cmdline

In [ ]:
%%cmd
%Temp%\tmp_bash.bat

In [ ]:
## Print processing time for the full process
print_processing_time(begintime_blockextraction_full, "Urban blocks extracted in: ")